<a href="https://colab.research.google.com/github/Atfssene/FRASA/blob/main/IndoSum_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IndoSum Dataset EDA**
For Bangkit Capstone project our ***Potato Tech*** team need to build machine learning model for Frasa to predict a summary from a sentences.

In this lab, we will do exploratory data analysis and preprocess the dataset.
The dataset is consists of news articles and manually constructed summaries [IndoSum dataset](https://drive.google.com/file/d/1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco/view) from [Kurniawan, Kemal & Louvan, Samuel (2018)](https://arxiv.org/abs/1810.05334).

Let's start!

## Preparation

In [ ]:
!pip install Sastrawi

     |████████████████████████████████| 215kB 15.0MB/s 


In [ ]:
# Import library
import pandas as pd
import json

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize.treebank import TreebankWordDetokenizer as untoken


## Load data

In [ ]:
# Download the data from drive
!gdown --id 1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco

Downloading...
From: https://drive.google.com/uc?id=1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco
To: /content/indosum.tar.gz
96.0MB [00:00, 126MB/s]


In [ ]:
# unzip

!tar -xvf /content/indosum.tar.gz

indosum/
indosum/test.02.jsonl
indosum/README.txt
indosum/train.05.jsonl
indosum/train.04.jsonl
indosum/test.04.jsonl
indosum/train.01.jsonl
indosum/test.05.jsonl
indosum/test.03.jsonl
indosum/test.01.jsonl
indosum/dev.04.jsonl
indosum/dev.01.jsonl
indosum/dev.02.jsonl
indosum/train.02.jsonl
indosum/dev.03.jsonl
indosum/dev.05.jsonl
indosum/CHANGELOG.txt
indosum/train.03.jsonl


In [ ]:
# We will use the first-fold data (train, dev, test)

url = "/content/indosum/train.01.jsonl"
with open(url, 'r') as f:
    data = f.readlines()
    train = list(map(json.loads, data)) 

train = pd.DataFrame(train)


## Exploratoratory

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14262 entries, 0 to 14261
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     14262 non-null  object
 1   gold_labels  14262 non-null  object
 2   id           14262 non-null  object
 3   paragraphs   14262 non-null  object
 4   source       14262 non-null  object
 5   source_url   14262 non-null  object
 6   summary      14262 non-null  object
dtypes: object(7)
memory usage: 780.1+ KB


Now we know the dataset consist of 7 columns and probably we will just take some of the column that related to this summarization work.

To make we sure what columns should we choose, let's take a look at some rows.

In [ ]:
train.head()

,category,gold_labels,id,paragraphs,source,source_url,summary
0,tajuk utama,"[[False, True], [True, True], [False, False, F...",1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dokter, R...",cnn indonesia,https://www.cnnindonesia.com/hiburan/201708041...,"[[Dokter, Lula, Kamal, yang, merupakan, selebr..."
1,teknologi,"[[False, False, False, False], [False, True, T...",1509072914-dua-smartphone-zenfone-baru-tawarka...,"[[[Selfie, ialah, salah, satu, tema, terpanas,...",dailysocial.id,https://dailysocial.id/post/dua-smartphone-zen...,"[[Asus, memperkenalkan, , ZenFone, generasi, ..."
2,hiburan,"[[True], [True], [False, False], [False], [Fal...",1510613677-songsong-visit-2020-bengkulu-perkua...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dinas, Pa...",cnn indonesia,https://www.cnnindonesia.com/gaya-hidup/201711...,"[[Dinas, Pariwisata, Provinsi, Bengkulu, kemba..."
3,tajuk utama,"[[True, True], [False, False, False], [True], ...",1502706803-icw-ada-kejanggalan-atas-tewasnya-s...,"[[[Merdeka.com, -, Indonesia, Corruption, Watc...",merdeka,https://www.merdeka.com/peristiwa/icw-merasa-a...,"[[Indonesia, Corruption, Watch, (, ICW, ), mem..."
4,tajuk utama,"[[False, True], [True, True, True], [False], [...",1503039338-pembagian-sepeda-usai-upacara-penur...,"[[[Merdeka.com, -, Presiden, Joko, Widodo, (, ...",merdeka,https://www.merdeka.com/peristiwa/usai-upacara...,"[[Jokowi, memimpin, upacara, penurunan, bender..."


Wow... that's a unique dataset we have here!

1. The news content and summary are raw text but tokenized.
2. Punctuation has not been removed, also with that capital letter.
3. Column gold_labels is a boolean list of main idea sentences in paragraphs.

From this result, the columns will be used are **gold_labels, paragraphs**, and **summary**.

In [ ]:
df_train = train[['gold_labels', 'paragraphs', 'summary']].copy()  

df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14262 entries, 0 to 14261
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gold_labels  14262 non-null  object
 1   paragraphs   14262 non-null  object
 2   summary      14262 non-null  object
dtypes: object(3)
memory usage: 334.4+ KB


,gold_labels,paragraphs,summary
0,"[[False, True], [True, True], [False, False, F...","[[[Jakarta, ,, CNN, Indonesia, -, -, Dokter, R...","[[Dokter, Lula, Kamal, yang, merupakan, selebr..."
1,"[[False, False, False, False], [False, True, T...","[[[Selfie, ialah, salah, satu, tema, terpanas,...","[[Asus, memperkenalkan, , ZenFone, generasi, ..."
2,"[[True], [True], [False, False], [False], [Fal...","[[[Jakarta, ,, CNN, Indonesia, -, -, Dinas, Pa...","[[Dinas, Pariwisata, Provinsi, Bengkulu, kemba..."
3,"[[True, True], [False, False, False], [True], ...","[[[Merdeka.com, -, Indonesia, Corruption, Watc...","[[Indonesia, Corruption, Watch, (, ICW, ), mem..."
4,"[[False, True], [True, True, True], [False], [...","[[[Merdeka.com, -, Presiden, Joko, Widodo, (, ...","[[Jokowi, memimpin, upacara, penurunan, bender..."


Let's check if we had empty value.

In [ ]:
df_train.isnull().any()

gold_labels    False
paragraphs     False
summary        False
dtype: bool

Great!

No empty value here from **14262 rows** data.

That's mean we can start preprocessing the dataset.
For text-type dataset, we need to do some steps:
1. Cleaning punctuation
2. Stemming
3. Remove stopwords
4. Lowercasing letter